In [1]:
import pandas as pd


In [2]:
def delete_duplicates_and_save(input_file, inputs_duplicates, inputs_na):
    df = pd.read_csv(input_file)
    df_unique = df.drop_duplicates(subset=inputs_duplicates)
    df_unique = df_unique.dropna(subset=inputs_na)
    df_unique.to_csv(input_file, index=False)

In [3]:
# delete_duplicates_and_save("SteamAppsList.csv",["appid", "name"], ["appid", "name"])

In [4]:
df = pd.read_csv("SteamPlayersCountHistory.csv")
df.info()
df = df.drop_duplicates(subset=["name", "appid"])
df.info()
df = df.dropna(subset=["date_playerscount","name"])
df.info()
df.count()
# df.to_csv("SteamPlayersCountHistory.csv", index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92865 entries, 0 to 92864
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   appid              92865 non-null  int64 
 1   name               92865 non-null  object
 2   date_playerscount  92865 non-null  object
dtypes: int64(1), object(2)
memory usage: 2.1+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92865 entries, 0 to 92864
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   appid              92865 non-null  int64 
 1   name               92865 non-null  object
 2   date_playerscount  92865 non-null  object
dtypes: int64(1), object(2)
memory usage: 2.1+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92865 entries, 0 to 92864
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   appid  

appid                92865
name                 92865
date_playerscount    92865
dtype: int64

In [5]:
# delete_duplicates_and_save("SteamPlayersCountHistory.csv", ["appid", "name"], ["appid", "name", "date_playerscount"])

In [11]:
df = pd.read_csv("SteamPlayersCountHistory.csv")

games_that_dont_have_players = []

now = 1749234827 * 1000
one_day_ago = now - 86400000

for data in df.iterrows():
    split_to_last = data[1]["date_playerscount"].split(", ")[-1]
    current_data_split = split_to_last.split(" ")
    timestamp_ms = int(current_data_split[0])
    if not (one_day_ago <= timestamp_ms <= now):
        games_that_dont_have_players.append(data[1]["appid"])
        continue

    all_players = []
    for current_data in data[1]["date_playerscount"].split(", "):
        current_data_split = current_data.split(" ")
        players_count = int(current_data_split[1])
        all_players.append(players_count)
     
    
    if max(all_players) <= 3:
        games_that_dont_have_players.append(data[1]["appid"])



In [12]:
df = df[~df["appid"].isin(games_that_dont_have_players)]
df.info()
df.to_csv("Clean_SteamHistory_PCout.csv", index=False)

<class 'pandas.core.frame.DataFrame'>
Index: 9691 entries, 5 to 92857
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   appid              9691 non-null   int64 
 1   name               9691 non-null   object
 2   date_playerscount  9691 non-null   object
dtypes: int64(1), object(2)
memory usage: 302.8+ KB
